In [ ]:
import myproject

import pandas as pd
import json
import numpy as np

In [ ]:
from myproject.dcf.all_transforms import Transform
from myproject.dcf.lispy import s

In [ ]:
df = pd.read_csv('./problem.csv')
df2 = pd.read_csv('../dcf-server/sample_data/yellow_tripdata_2021-02.csv')

In [ ]:
w = myproject.ExampleWidget(df=df)
w

In [ ]:
@w.add_command
class GroupBy2(Transform):
    command_default = [s("groupby2"), s('df'), 'col', {}]
    command_pattern = [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]
    @staticmethod 
    def transform(df, col, col_spec):
        grps = df.groupby(col)
        df_contents = {}
        for k, v in col_spec.items():
            if v == "sum":
                df_contents[k] = grps[k].apply(lambda x: x.sum())
            elif v == "mean":
                df_contents[k] = grps[k].apply(lambda x: x.mean())
            elif v == "median":
                df_contents[k] = grps[k].apply(lambda x: x.median())
            elif v == "count":
                df_contents[k] = grps[k].apply(lambda x: x.count())
        return pd.DataFrame(df_contents)

    #test_df = group_df
    #test_sequence = [s("groupby2"), s('df'), 'c', dict(a='sum', b='mean')]
    test_output = pd.DataFrame(
        {'a':[100, 110], 'b':[2.5, 5.5]},
        index=['q','w'])

    @staticmethod 
    def transform_to_py(df, col, col_spec):
        commands = [
            "    grps = df.groupby('%s')" % col,
            "    df_contents = {}"
        ]
        for k, v in col_spec.items():
            if v == "sum":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.sum())" % (k, k))
            elif v == "mean":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.mean())" % (k, k))
            elif v == "median":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.median())" % (k, k))
            elif v == "count":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.count())" % (k, k))
        #print("commands", commands)
        commands.append("    df = pd.DataFrame(df_contents)")
        return "\n".join(commands)


In [ ]:
GroupBy2.__name__

In [ ]:
small_command_config = {'commandPatterns': {'fillna': [[3, 'fillVal', 'type', 'integer']],
  'dropcol': [None],
  'onehot': [None]},
 'commandDefaults': {'fillna': [{'symbol': 'fillna'},
   {'symbol': 'df'},
   'col',
   8],
  'dropcol': [{'symbol': 'dropcol'}, {'symbol': 'df'}, 'col'],
  'onehot': [{'symbol': 'onehot'}, {'symbol': 'df'}, 'col']}}
w.command_config = small_command_config

In [ ]:
orig_command_config = {'commandPatterns': {'fillna': [[3, 'fillVal', 'type', 'integer']],
  'dropcol': [None],
  'onehot': [None],
  'groupby': [[3,
    'colMap',
    'colEnum',
    ['null', 'sum', 'mean', 'median', 'count']]]},
 'commandDefaults': {'fillna': [{'symbol': 'fillna'},
   {'symbol': 'df'},
   'col',
   8],
  'dropcol': [{'symbol': 'dropcol'}, {'symbol': 'df'}, 'col'],
  'onehot': [{'symbol': 'onehot'}, {'symbol': 'df'}, 'col'],
  'groupby': [{'symbol': 'groupby'}, {'symbol': 'df'}, 'col', {}]}}

In [ ]:
w.command_config = orig_command_config

In [ ]:
df2

In [ ]:
w.transform_error

In [ ]:
w2 = myproject.ExampleWidget(df2)
w2

In [ ]:
w

In [ ]:
w.transformed_df = json.loads(df.to_json(orient='table', indent=2))

In [ ]:
w.transformed_df.on?

In [ ]:
from myproject.dcf import 

In [ ]:
w.value

In [ ]:
w.value2

In [ ]:
w.commands

In [ ]:
assert w.value == 'Hello World'

In [ ]:
w.value = "asdfasdf"